In [150]:
import aoc.common.process_input
import aoc.year2022.day15

In [151]:
fileContent = aoc.common.process_input.read_file("input\\year2022\\day15\\example.txt")        
input_list = aoc.common.process_input.to_function_list(fileContent, str)

In [152]:
# Sensor at x=2, y=18: closest beacon is at x=-2, y=15
def process_line(line):
    parts = line.split(' ')

    sensor_x_str = parts[2]
    sensor_x = int(sensor_x_str[2:-1])
    sensor_y_str = parts[3]
    sensor_y = int(sensor_y_str[2:-1])

    beacon_x_str = parts[8]
    beacon_x = int(beacon_x_str[2:-1])
    beacon_y_str = parts[9]
    beacon_y = int(beacon_y_str[2:])
    
    return (sensor_x, sensor_y), (beacon_x, beacon_y)

process_line("Sensor at x=2, y=18: closest beacon is at x=-2, y=15")

((2, 18), (-2, 15))

In [153]:
sensor_beacon_pairs = []
for input in input_list:
    sensor, beacon = process_line(input)
    sensor_beacon_pairs.append((sensor, beacon))

y_line = 0 # for the example, that's another thing we need to make sure it is passed

# need to check if the circle with sensor as center, and radius the vector "sensor-beacon" intersects line Y=YLINE

cross_pairs_and_md = []
for sbp in sensor_beacon_pairs:
    s = sbp[0]
    b = sbp[1]
    md = abs(s[0]-b[0]) + abs(s[1]-b[1])
    if s[1] < y_line:
        if s[1] + md > y_line:
            # print(sbp, "crosses line", y_line, "distance", md)
            cross_pairs_and_md.append((sbp[0], sbp[1], md))
    elif s[1] > y_line:
        if s[1] - md < y_line:
            # print(sbp, "crosses line", y_line, "distance", md)
            cross_pairs_and_md.append((sbp[0], sbp[1], md))
    else: # s[1] == y_line
        # print(sbp, "crosses line", y_line, "distance", md)
        cross_pairs_and_md.append((sbp[0], sbp[1], md))

print(cross_pairs_and_md)

intersect_x = []
intersect_x_pairs = []
for cp in cross_pairs_and_md:
    s = cp[0]
    b = cp[1]
    md = cp[2]

    px_1 = - (md - abs(y_line - s[1])) + s[0]
    px_2 = (md - abs(y_line - s[1])) + s[0]
    intersect_x.append(px_1)
    intersect_x.append(px_2)
    intersect_x_pairs.append((px_1, px_2))
    
intersect_x.sort()
print(intersect_x_pairs)
min_x = intersect_x[0]
max_x = intersect_x[-1]

[((13, 2), (15, 3), 3), ((8, 7), (2, 10), 9), ((2, 0), (2, 10), 10), ((20, 1), (15, 3), 7)]
[(12, 14), (6, 10), (-8, 12), (14, 26)]


In [154]:
beacons_in_y_line = []
for sbp in sensor_beacon_pairs:
    b = sbp[1]
    if b[1] == y_line:
        if b[0] not in beacons_in_y_line:
            beacons_in_y_line.append(b[0])

print(beacons_in_y_line)

[]


In [155]:
result = abs(min_x) + abs(max_x) + 1 - len(beacons_in_y_line)
print(result)

35


In [165]:
# md = abs(point[x] - sensor[x]) + abs(point[y] - sensor[y])
# tipping point is when x stops increasing and reaches its max value
# max_x = sensor[0] + md
# what's the y?
# md = abs((sensor[x] + md) - sensor[x]) + abs(point[y] - sensor[y])
# md = abs(sensor[x] + md - sensor[x]) + abs(point[y] - sensor[y])
# md = md + abs(point[y] - sensor[y])
# 0 = abs(point[y] - sensor[y])
# point[y] = sensor[y]

def check_intercepts(storage_dict, y_line):
    
    for i in storage_dict.keys():
        if storage_dict[i]["min_y"] <= y_line and storage_dict[i]["max_y"] >= y_line:
            # intersects
            if y_line == storage_dict[i]["min_y"]:
                if len(storage_dict[i]["ip"]) > 0: # sanity check
                    raise Exception("interception points should be empty at " + y_line + " for " + i)

                point = (storage_dict[i]["s"][0], y_line)
                storage_dict[i]["ip"].append(point) # there's only one point

            elif y_line == (storage_dict[i]["min_y"] + 1): # one point becomes two
                current_list = storage_dict[i]["ip"]
                storage_dict[i]["ip"] = []
                storage_dict[i]["ip"].append((current_list[0][0]-1, y_line))
                storage_dict[i]["ip"].append((current_list[0][0]+1, y_line))

            elif y_line > storage_dict[i]["min_y"] and y_line <= storage_dict[i]["tp"]: # until tipping point
                current_list = storage_dict[i]["ip"]
                storage_dict[i]["ip"] = []
                storage_dict[i]["ip"].append((current_list[0][0]-1, y_line))
                storage_dict[i]["ip"].append((current_list[1][0]+1, y_line))

            elif y_line > storage_dict[i]["tp"] and y_line < storage_dict[i]["max_y"]: # until max_y
                current_list = storage_dict[i]["ip"]
                storage_dict[i]["ip"] = []
                storage_dict[i]["ip"].append((current_list[0][0]+1, y_line))
                storage_dict[i]["ip"].append((current_list[1][0]-1, y_line))

            elif y_line == storage_dict[i]["max_y"]: # two points become 1
                current_list = storage_dict[i]["ip"]
                storage_dict[i]["ip"] = []
                storage_dict[i]["ip"].append((current_list[0][0]+1, y_line))

            else:
                raise Exception("something went wrong")
                
        else: # doesn't intersect
            storage_dict[i]["ip"] = []


def find_gaps_in_intercepts(storage):
    intersect = []
    for key in storage.keys():
        if len(storage[key]["ip"]) > 0:
            intersect.append(storage[key]["ip"])
    if len(intersect) == 0:
        return False, 0

    intersect.sort(key=lambda a: a[0])
    print(intersect)

    first = intersect.pop(0)
    if len(first) > 1:
        min_x = first[0][0]
        max_x = first[1][0]
    else:
        min_x = first[0][0]
        max_x = min_x

    # keep within the bounds
    min_x = max(min_x, 0)
    max_x = min(max_x, 20)

    while len(intersect) > 0:
        print(intersect, min_x, max_x)
        item = intersect.pop(0)
        
        new_x = item[0][0]
        if new_x > min_x and new_x < max_x: # within our interval already
            new_x = new_x # nop
        elif new_x < min_x and new_x > 0: # it will never pass here because i is sorted
            if min_x - new_x > 1:
                # that's a gap
                return True, new_x
            else: 
                min_x = new_x # consecutive
        elif new_x > max_x and new_x < 20:
            if new_x - max_x > 1:
                # that's a gap
                return True, new_x
            else:
                max_x = new_x
        if len(item) > 1:
            new_x = item[1][0]
            if new_x > min_x and new_x < max_x: # within our interval already
                new_x = new_x # nop
            elif new_x < min_x and new_x > 0: # it will never pass here because i is sorted
                if min_x - new_x > 1:
                    # that's a gap
                    return True, new_x
                else: 
                    min_x = new_x # consecutive
            elif new_x > max_x and new_x < 20:
                    max_x = new_x # there's no chance for gap becase, the first one was within, this just extends the thing

    return False, 0


fileContent = aoc.common.process_input.read_file("input\\year2022\\day15\\example.txt")        
input_list = aoc.common.process_input.to_function_list(fileContent, str)

storage = {}
beacons = []
for i in range(len(input_list)):
    record = {}
    s, b = process_line(input_list[i])
    md = abs(s[0]-b[0]) + abs(s[1]-b[1])
    record["s"] = s
    record["b"] = b
    record["md"] = md
    record["min_y"] = s[1] - md
    record["max_y"] = s[1] + md
    record["tp"] = s[1] # tipping point, max_x
    record["ip"] = [] # intersection points for the current y_line
    storage[i] = record
    if b not in beacons:
        beacons.append(b)

    # if there is intersections in line 0, need to init
    if record["min_y"] <= 0:
        px_1 = - (md - abs(0 - s[1])) + s[0]
        px_2 = (md - abs(0 - s[1])) + s[0]
        record["ip"].append((px_1, 0))
        if px_1 != px_2:
            record["ip"].append((px_2, 0))
    

min_line = 0
max_line = 20 # changes for input

# for key in storage.keys():
#     if len(storage[key]["ip"]) > 0:
#         print(key, storage[key])

for y in range(min_line+1, max_line+1):
    check_intercepts(storage, y)
    has_gaps, x = find_gaps_in_intercepts(storage)
    if has_gaps:
        # is the gap a beacon?
        if (x, y) in beacons:
            continue
        else:
            print(x, y)
            break

print(beacons)
# for key in storage.keys():
#     if len(storage[key]["ip"]) > 0:
#         print(key, storage[key])

[[(-7, 1), (11, 1)], [(5, 1), (11, 1)], [(11, 1), (15, 1)], [(13, 1), (27, 1)]]
[[(5, 1), (11, 1)], [(11, 1), (15, 1)], [(13, 1), (27, 1)]] 0 11
[[(11, 1), (15, 1)], [(13, 1), (27, 1)]] 0 11
[[(13, 1), (27, 1)]] 0 15
[[(-6, 2), (10, 2)], [(4, 2), (12, 2)], [(10, 2), (16, 2)], [(14, 2)], [(14, 2), (26, 2)], [(16, 2)]]
[[(4, 2), (12, 2)], [(10, 2), (16, 2)], [(14, 2)], [(14, 2), (26, 2)], [(16, 2)]] 0 10
[[(10, 2), (16, 2)], [(14, 2)], [(14, 2), (26, 2)], [(16, 2)]] 0 12
[[(14, 2)], [(14, 2), (26, 2)], [(16, 2)]] 0 16
[[(14, 2), (26, 2)], [(16, 2)]] 0 16
[[(16, 2)]] 0 16
[[(-5, 3), (9, 3)], [(3, 3), (13, 3)], [(11, 3), (15, 3)], [(13, 3), (15, 3)], [(15, 3), (17, 3)], [(15, 3), (25, 3)]]
[[(3, 3), (13, 3)], [(11, 3), (15, 3)], [(13, 3), (15, 3)], [(15, 3), (17, 3)], [(15, 3), (25, 3)]] 0 9
[[(11, 3), (15, 3)], [(13, 3), (15, 3)], [(15, 3), (17, 3)], [(15, 3), (25, 3)]] 0 13
[[(13, 3), (15, 3)], [(15, 3), (17, 3)], [(15, 3), (25, 3)]] 0 15
[[(15, 3), (17, 3)], [(15, 3), (25, 3)]] 0 15
[[(

In [159]:
# the idea for part 2
# for each input line, calculate md and save it
# for each y
# calculate the intersections
# after the first line, x goes up or down by 1, depending if I passed the further's x (equal to md)
# need to change the calculation for the gap

def find_gaps_in_intercepts_inner(intersect):

    intersect.sort(key=lambda a: a[0])
    print(intersect)

    first = intersect.pop(0)
    if len(first) > 1:
        min_x = first[0][0]
        max_x = first[1][0]
    else:
        min_x = first[0][0]
        max_x = min_x

    # keep within the bounds
    min_x = max(min_x, 0)
    max_x = min(max_x, 20)

    while len(intersect) > 0:
        print(intersect, min_x, max_x)
        item = intersect.pop(0)
        
        new_x = item[0][0]
        if new_x > min_x and new_x < max_x: # within our interval already
            new_x = new_x # nop
        elif new_x < min_x and new_x > 0: # it will never pass here because i is sorted
            if min_x - new_x > 1:
                # that's a gap
                return True, new_x
            else: 
                min_x = new_x # consecutive
        elif new_x > max_x and new_x < 20:
            if new_x - max_x > 1:
                # that's a gap
                return True, new_x
            else:
                max_x = new_x
        if len(item) > 0:
            new_x = item[1][0]
            if new_x > min_x and new_x < max_x: # within our interval already
                new_x = new_x # nop
            elif new_x < min_x and new_x > 0: # it will never pass here because i is sorted
                if min_x - new_x > 1:
                    # that's a gap
                    return True, new_x
                else: 
                    min_x = new_x # consecutive
            elif new_x > max_x and new_x < 20:
                    max_x = new_x # there's no chance for gap becase, the first one was within, this just extends the thing

    return False, 0

find_gaps_in_intercepts_inner([[(-7, 1), (11, 1)], [(5, 1), (11, 1)], [(11, 1), (15, 1)], [(13, 1), (27, 1)]])

[[(-7, 1), (11, 1)], [(5, 1), (11, 1)], [(11, 1), (15, 1)], [(13, 1), (27, 1)]]
[[(5, 1), (11, 1)], [(11, 1), (15, 1)], [(13, 1), (27, 1)]] 0 11
[[(11, 1), (15, 1)], [(13, 1), (27, 1)]] 0 11
[[(13, 1), (27, 1)]] 0 15


(False, 0)